#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from google.colab import drive
import os

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="124M") # switch to medium model (or better if colab is upgraded)

Fetching checkpoint: 1.05Mit [00:00, 420Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 95.9Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 805Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:08, 55.6Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 706Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 135Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 155Mit/s]                                                       


## Mounting Google Drive and Change Working Directory


> The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

> Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import files for fine tuning 

In [0]:
# Change the current working directory
transcript_file_location = "processed_transcripts.txt"

In [0]:
print(os.getcwd())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-efacdae2ab0b>", line 1, in <module>
    print(os.getcwd())
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/

OSError: ignored

In [0]:
transcripts = gpt2.copy_file_from_gdrive(transcript_file_location)

FileNotFoundError: ignored

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:04<00:00,  4.82s/it]


dataset has 1096225 tokens
Training...
[10 | 38.44] loss=2.99 avg=2.99
[20 | 70.46] loss=2.70 avg=2.84
[30 | 102.56] loss=2.80 avg=2.83
[40 | 134.65] loss=2.82 avg=2.83
[50 | 166.85] loss=2.81 avg=2.82
[60 | 199.01] loss=2.63 avg=2.79
[70 | 231.25] loss=2.48 avg=2.75
[80 | 263.62] loss=2.76 avg=2.75
[90 | 296.02] loss=2.85 avg=2.76
[100 | 328.39] loss=2.47 avg=2.73
[110 | 360.78] loss=2.59 avg=2.71
[120 | 393.22] loss=2.66 avg=2.71
[130 | 425.58] loss=2.57 avg=2.70
[140 | 457.99] loss=2.40 avg=2.68
[150 | 490.39] loss=2.75 avg=2.68
[160 | 522.74] loss=2.70 avg=2.68
[170 | 555.09] loss=2.75 avg=2.69
[180 | 587.52] loss=2.58 avg=2.68
[190 | 619.88] loss=2.64 avg=2.68
[200 | 652.26] loss=2.80 avg=2.69
======== SAMPLE 1 ========
 and like yeah. It's all very weird. It's so different than when I was younger. It's like like the same sort of...but I was just... I'm not a kid or an artist, I'm not even a poet. I'm like I have this thing, and it's this way. It's all these pieces, you know? Anyw

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='run1')

FailedPreconditionError: ignored

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
input_text="""CLIENT: I keep worrying at night. I just can't sleep. What should I do?
THERAPIST:"""

numberofsamples=10

import time

t0=time.time()

single_text=gpt2.generate(sess,
              length=60,
              temperature=.4,
              prefix=input_text,
              nsamples=numberofsamples,
              batch_size=1,
              return_as_list=True
              )

t1=time.time()
total=t1-t0
print(total)

#print(single_text)
for i in range(0,(numberofsamples-1)):
  print(single_text[i].split("THERAPIST:")[1].split("CLIENT:")[0])



11.747133493423462
 It's not that you can't sleep, it's that you can't.	

 You can't.	

 You're worried?	

 You know, it's not a question of, you know, this is what you're going to do. You know, you're going to do what you're going to do. You know, you're going to do what you're going to do. You know, it's not going to
 You know, that's what I'm trying to say. That's what I'm trying to get at. That's what I'm trying to get at. I'm trying to figure out what's what.	

 You can't.	

 Mm hm.	

 You know, it sounds like you're feeling some kind of anxiety about your work.	

 What do you think?	



**Create the chatbot!**

In [0]:
input_text=""
chatbot_response=""
numberofsamples=1
i=0
oldtext=""
temp=.8
while input_text!="ABORT":
  i=i+1
  if i==1:
    print("Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.")
  input_text = input("\n")
  if input_text=="ABORT":
    break
    

  combinedtext=oldtext+" "+"CLIENT: "+input_text+"\nTHERAPIST:"
  single_text=gpt2.generate(sess,
              length=120,
              temperature=temp,
              prefix=combinedtext,
              nsamples=numberofsamples,
              batch_size=1,
              return_as_list=True
              )
  
  #Go Through the batch and see which if the response is greater than 15 characters
#  for ii in range(0,(numberofsamples-1)):
#    if (len(single_text[ii].split("THERAPIST:")[1].split("CLIENT:")[0])>15):
#      single_text[0]=single_text[ii]
#      break

  temp=.4
  while (len(single_text[0].split("THERAPIST:")[i].split("CLIENT:")[0])<15):
    temp=temp+.05
    single_text=gpt2.generate(sess,
              length=120,
              temperature=temp,
              prefix=combinedtext,
              nsamples=numberofsamples,
              batch_size=1,
              return_as_list=True
              )
  
  oldtext=combinedtext+single_text[0].split("THERAPIST: ")[1].split("CLIENT:")[0]
  
  print("\n"+single_text[0].split("THERAPIST:")[i].split("CLIENT:")[0]) #change 1 to i if the prior is re-insurted 


Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.

I've had a very busy day. The winter is making me feel down.

 Well, I mean maybe we're not talking about job stuff necessarily, but like you said yesterday, it's been a busy week. I mean you've been going over the latest job listings, and it looked like you were leaving a little bit earlier today and leaving a bit later today. So it's been a busy week. And I think it's been a busy week also. I think you've been busy in terms of, you know, just kind of your work/life stuff. You've been really busy with music. You've had a lot of interesting projects. But I think I

I've really been applying to jobs.

 Yeah. That you really should be applying to. And it seems like you've been applying to a lot of different things. Like you're applying to jobs that are really important to you that are not easy to leave alone. Like you're applying to jobs that are really important to you that are 

In [0]:
print(single_text[0])



For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.